# Homework 10

* **Build the best model on the kaggle [Housing Data](https://www.kaggle.com/c/house-prices-advanced-regression-techniques) that you can !**

* **Use only Ridge or lasso or elastic net.**

* **Upload a screenshot of your leaderboard ranking**

**import libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('../data/kaggle/HousePrices/train.csv')

In [ ]:
#Submission:
pred_df = pd.DataFrame(y_pred, index=df_test.index, columns=["SalePrice"])
pred_df.to_csv('../data/kaggle/HousePrices/submissions/en1.csv', header=True, index_label='Id')